In [48]:
import numpy as np
import pandas as pd

import joblib
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.base import BaseEstimator, TransformerMixin

In [49]:
data = pd.read_csv('salaries.csv')

In [50]:
model = GradientBoostingRegressor(**{'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100})

In [51]:
X = data.drop('salary_in_usd', axis=1)
y = data['salary_in_usd']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [52]:
data

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M
...,...,...,...,...,...,...,...,...,...,...,...
3750,2020,SE,FT,Data Scientist,412000,USD,412000,US,100,US,L
3751,2021,MI,FT,Principal Data Scientist,151000,USD,151000,US,100,US,L
3752,2020,EN,FT,Data Scientist,105000,USD,105000,US,100,US,S
3753,2020,EN,CT,Business Data Analyst,100000,USD,100000,US,100,US,L


In [53]:
variables_categoricas = ['experience_level', 'employment_type', 'employee_residence', 'remote_ratio', 'company_location', 'company_size']
job_title = ['job_title']

In [54]:
# Crear una clase para ejecutar el LabelEncoder
class LabelEncoderTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.encoder = LabelEncoder()

    def fit(self, X, y=None):
        self.encoder.fit(y)
        return self

    def transform(self, X, y=None):
        return self.encoder.transform(y)


In [55]:
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, variables_categoricas),
        ('job_title', LabelEncoderTransformer(), job_title)
    ])

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', model)
])


In [56]:
pipeline.fit(X_train, y_train)

TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'

In [58]:
import numpy as np
import pandas as pd
import joblib
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.base import BaseEstimator, TransformerMixin

# Cargar los datos
data = pd.read_csv('salaries.csv')

# Definir el modelo
model = GradientBoostingRegressor(**{'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100})

# Dividir los datos
X = data.drop('salary_in_usd', axis=1)
y = data['salary_in_usd']

# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir las columnas
variables_categoricas = ['experience_level', 'employment_type', 'employee_residence', 'remote_ratio', 'company_location', 'company_size']
job_title = ['job_title']

class LabelEncoderTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.encoder = LabelEncoder()

    def fit(self, X, y=None):
        # Usar ravel() para asegurar que y sea unidimensional
        self.encoder.fit(y.ravel())
        return self

    def transform(self, X, y=None):
        # Usar ravel() también aquí
        return self.encoder.transform(y.ravel())


# Crear el pipeline de preprocesamiento para las columnas categóricas
categorical_transformer = Pipeline(steps=[
    ('label_encoder', LabelEncoderTransformer())
])

# Transformador para las variables categóricas
preprocessor = ColumnTransformer(
    transformers=[
        ('job_title', categorical_transformer, job_title), 
        ('cat_vars', OneHotEncoder(), variables_categoricas)
    ])

# Crear el pipeline completo
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', model)
])

# Entrenar el modelo
pipeline.fit(X_train, y_train)

# Evaluar el modelo
y_pred = pipeline.predict(X_test)

# Imprimir métricas
print(f'R^2: {r2_score(y_test, y_pred)}')
print(f'Mean Absolute Error: {mean_absolute_error(y_test, y_pred)}')
print(f'Mean Squared Error: {mean_squared_error(y_test, y_pred)}')


/var/folders/h9/bqqwl8l10jb5klwskmyw_0r40000gn/T/ipykernel_19529/1478804869.py:36: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  self.encoder.fit(y.ravel())


AttributeError: 'NoneType' object has no attribute 'ravel'

In [ ]:
y_pred = pipeline.predict(X_test)

r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)

print(f'R2: {r2}')
print(f'RMSE: {rmse}')
print(f'MAE: {mae}')

In [ ]:
joblib.dump(pipeline, 'pipeline.joblib')